In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import json
import geopandas as gpd
import utils
from utils import util_functions

In [2]:
df, twi_df = util_functions.get_processed_merged_data()

c:\src\jupyter_codes\foreign-affairs\utils\util_functions.py:26: FutureWarning: Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`
  country_list = df['focus group'][idx]


## Rest

In [ ]:
df.columns

In [3]:
df.head(1)

,Region of Focus,Language,Entity owner (English),Parent entity (English),X (Twitter) handle,X (Twitter) URL,X (Twitter) Follower #,Facebook page,Facebook URL,Facebook Follower #,...,TikTok Subscriber #,Name (English),focus group,is_blue_verified,created_at,followers_count,following_count,tweet_num,followers_following_ratio,formatted_created_at
0,Anglosphere,English,China Media Group (CMG),Central Publicity Department,_bubblyabby_,https://twitter.com/_bubblyabby_,1678.0,itsAbby-103043374799622,https://www.facebook.com/itsAbby-103043374799622,1387432.0,...,660.0,Yang Xinmeng (Abby Yang),"[Australia, Canada, New Zealand, United Kingdo...",True,2014-07-10 02:05:08,2799.0,161.0,851.0,17.385093,2014-07-10


In [4]:
temp_series = df['is_blue_verified'].value_counts(dropna=False)
temp_df = pd.DataFrame({'Name': ["Is Verified", "Is Not Verified", "N/A"], 'Value': list(temp_series.values)})
temp_df

,Name,Value
0,Is Verified,443
1,Is Not Verified,204
2,N/A,111


In [5]:
list(temp_series.index)

[False, nan, True]

In [6]:
temp_df = pd.DataFrame({'Name': [], 'Value': []})